# 📘 RAG Evaluation Assignment

## 🎯 Objective
This notebook evaluates the performance of a Retrieval-Augmented Generation (RAG) pipeline using local models and semantic similarity metrics.

We use cosine similarity between embeddings of:
- Queries
- Contexts
- Generated Answers

to compute:
- **Faithfulness**
- **Answer Relevancy**


##  Step 1: Import Required Libraries

We use `sentence-transformers` for embeddings, `torch` for tensor operations, and `json` for loading and saving evaluation logs.


In [1]:
import json
import torch
from sentence_transformers import SentenceTransformer, util


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


##  Step 2: Load Embedding Model

I have loaded `BAAI/bge-large-en-v1.5` to compute semantic embeddings for queries, contexts, and answers. The model runs on GPU if available.


In [3]:
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device=device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [4]:
BATCH_SIZE = 16


##  Step 3: Load and Parse Evaluation Logs

We extract:
- User queries
- Retrieved context
- Assistant answers

from the `logs.json` file to prepare for scoring.


In [5]:
with open("logs.json", "r", encoding="utf-8") as f:
    data = json.load(f)


##  Step 4: Generate Embeddings

Embeddings are computed for:
- `contexts` → for **faithfulness**
- `queries` → for **relevancy**
- `answers` → as the generated output

These are used for cosine similarity evaluation.


In [6]:
ids = []
contexts, queries, answers = [], [], []

for block in data:
    for item in block.get("items", []):
        ids.append(item["id"])

        system_prompt, user_query = "", ""
        for inp in item["input"]:
            if inp["role"] == "system":
                system_prompt = inp["context"]
            elif inp["role"] == "user":
                user_query = inp["context"]

        assistant_response = " ".join(out["content"] for out in item["expectedOutput"])

        contexts.append(system_prompt)
        queries.append(user_query)
        answers.append(assistant_response)


In [7]:
context_embs = model.encode(
    contexts,
    batch_size=BATCH_SIZE,
    convert_to_tensor=True,
    normalize_embeddings=True
)


In [8]:
query_embs = model.encode(
    queries,
    batch_size=BATCH_SIZE,
    convert_to_tensor=True,
    normalize_embeddings=True
)

In [9]:
answer_embs = model.encode(
    answers,
    batch_size=BATCH_SIZE,
    convert_to_tensor=True,
    normalize_embeddings=True
)

##  Step 5: Calculate Scores

Cosine similarity is used to compute:
- **Faithfulness** = similarity between `context` and `answer`
- **Answer Relevancy** = similarity between `query` and `answer`


In [10]:
results = []
for i, item_id in enumerate(ids):
    faithfulness_score = float(util.cos_sim(context_embs[i], answer_embs[i]))
    relevancy_score = float(util.cos_sim(query_embs[i], answer_embs[i]))

    results.append({
        "id": item_id,
        "faithfulness": round(faithfulness_score, 4),
        "answer_relevancy": round(relevancy_score, 4)
    })


##  Step 6: Save Evaluation Results

We write the final scores to `ragas_output.json` for further review and usage.


In [11]:
with open("ragas_output.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)


## ✅ Assignment Summary

This notebook presents a pipeline to evaluate RAG (Retrieval-Augmented Generation) outputs using semantic similarity. The objective was to assess how well retrieved contexts support generated answers across multiple user queries.

### 🔄 Approach Overview
- **Step 1**: Load logs containing queries, contexts, and answers.
- **Step 2**: Preprocess logs into evaluation-ready format.
- **Step 3**: Use **local models** (no paid APIs) to compute:
  - **Answer Relevancy**
  - **Faithfulness**
- **Step 4**: Output results in structured `.json` format.

### 🧰 Tools & Libraries Used
- `sentence-transformers`, `torch`, `json`
- Local models for fast, private, and cost-effective evaluation

### 📈 Outcome
- Successfully processed and scored all valid log entries
- Saved results in `ragas_output.json`

---
> ⚠️ Note: Any missing scores (e.g., NaNs) are likely due to incomplete input data or timeouts in some runs. These can be improved with better prompt handling or batching.
